In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import keras as k
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, MaxPool2D, Conv2D, Flatten
from keras.models import Model
from keras import initializers

Using TensorFlow backend.


In [3]:
import cv2
import glob
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pydicom
import pandas as pd
import pickle
import random
import sys

import data_manipulation
import pickle

In [4]:
with open("data/dataset_train.obj", "rb") as f:
    train = pickle.load(f)
with open("data/dataset_val.obj", "rb") as f:
    val = pickle.load(f)

In [5]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape = (1024, 1024, 3))

In [6]:
class_maxpool1 = MaxPool2D(pool_size=(2,2), strides = None, padding='same', name = 'classifier_maxpool1')(vgg16.layers[-1].output)
class_conv2d_1 = Conv2D(filters = 256, kernel_size=(1,1), padding='same', name='classifier_conv2d_1', 
                        activation='relu')(class_maxpool1)
class_conv2d_2 = Conv2D(filters = 56, kernel_size=(1,1), padding='same', name='classifier_conv2d_2', 
                        activation='relu')(class_conv2d_1)
class_conv2d_3 = Conv2D(filters = 28, kernel_size=(1,1), padding='same', name='classifier_conv2d_3', 
                        activation='relu')(class_conv2d_2)
flatten_1 = Flatten()(class_conv2d_3)
class_dense_1 = Dense(units = 2, activation ='softmax')(flatten_1)


my_model = Model(input=vgg16.input, output = class_dense_1)


c:\users\garna\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


my_model.summary()

In [7]:
my_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
train_labels = train.labels_list()
val_labels = val.labels_list()

In [9]:
import data_generator
params = {'dim': (1024,1024),
          'batch_size': 64,
          'n_classes': 2,
          'n_channels': 3,
          'shuffle': True}
training_generator = data_generator.DataGenerator(train, range(train.size()), train_labels, **params)
validation_generator = data_generator.DataGenerator(val, range(val.size()), val_labels, **params)

In [10]:
my_model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False)
# #my_model.train(train, val,
#             learning_rate=data_manipulation.LEARNING_RATE,
#             epochs=1,
#             layers='all',
#             augmentation=None)  ## no need to augment yet
# my_model.fit(x=train, output=,epochs=,batch_size=)

Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[64,1024,1024,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[Node: block1_conv1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_input_1_0_2, block1_conv1/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'block1_conv1/convolution', defined at:
  File "c:\users\garna\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\garna\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\garna\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_in